In [2]:
#coding=utf-8
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn import preprocessing  
min_max_scaler = preprocessing.MinMaxScaler()  
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import train_test_split

llb


In [19]:
data = pd.read_csv("../inputnew/contest_ext_crd_cd_ln_spl.csv")
print(data.shape)
print(data.columns)
data.head()

(67725, 6)
Index(['report_id', 'type_dw', 'get_time', 'changing_months',
       'changing_amount', 'content'],
      dtype='object')


,report_id,type_dw,get_time,changing_months,changing_amount,content
0,168790,3,2016-08-05,0,6068,0
1,169084,3,2016-08-27,-6,2000,0
2,1338584,3,2016-09-10,0,4542,0
3,1338584,3,2016-07-07,0,2001,0
4,1338584,3,2016-06-10,0,4560,0


In [20]:
print("data.isnull().sum()")
get_time_sec = []
import  time
for i in range(data.shape[0]):
    c = time.strptime(data['get_time'][i],"%Y-%m-%d")
    get_time_sec.append(int(time.mktime(c)))

data.isnull().sum()


In [21]:
data.get_time = get_time_sec
data.head()

,report_id,type_dw,get_time,changing_months,changing_amount,content
0,168790,3,1470326400,0,6068,0
1,169084,3,1472227200,-6,2000,0
2,1338584,3,1473436800,0,4542,0
3,1338584,3,1467820800,0,2001,0
4,1338584,3,1465488000,0,4560,0


In [22]:
# data.changing_months.unique()
data["REPORT_ID"] = data.report_id
del data["report_id"]
del data["changing_months"]
data.head()

,type_dw,get_time,changing_amount,content,REPORT_ID
0,3,1470326400,6068,0,168790
1,3,1472227200,2000,0,169084
2,3,1473436800,4542,0,1338584
3,3,1467820800,2001,0,1338584
4,3,1465488000,4560,0,1338584


In [25]:
def groupDeal(group):
    # print(group)
    type_dw_more = group.type_dw.value_counts().iloc[0]
    type_dw_less = group.type_dw.value_counts().iloc[-1]
    type_dw_num = group.type_dw.shape[0]
    
    content_more = group.content.value_counts().iloc[0]
    content_less = group.content.value_counts().iloc[-1]
    content_num = group.content.shape[0]

    get_time_max = group.get_time.max()
    get_time_min = group.get_time.min()
    get_time_mean = round(group.get_time.mean())
    get_time_zongshu = group.get_time.value_counts().iloc[0]
    get_time_num = group.get_time.shape[0]

    changing_amount_max = group.changing_amount.max()
    changing_amount_min = group.changing_amount.min()
    changing_amount_mean = group.changing_amount.mean()
    changing_amount_zongshu = group.changing_amount.value_counts().iloc[0]
    changing_amount_num = group.changing_amount.shape[0]


    indexs = {
        "REPORT_ID":int(group.REPORT_ID.iloc[0]),
        "type_dw_num": type_dw_num,
        "content_num": content_num,
        

        "get_time_max": get_time_max,
        "get_time_min": get_time_min,
        "get_time_mean": get_time_mean,
        "get_time_zongshu": get_time_zongshu,
        "get_time_num": get_time_num,

        "changing_amount_max": changing_amount_max,
        "changing_amount_min": changing_amount_min,
        "changing_amount_mean": changing_amount_mean,
        "changing_amount_zongshu": changing_amount_zongshu,
        "changing_amount_num": changing_amount_num,
        
        "type_dw_more": type_dw_more,
        "type_dw_less": type_dw_less,
        "content_more": content_more,
        "content_less": content_less
    }
    return pd.Series(data=[indexs[c] for c in indexs], index=[c for c in indexs])

In [26]:
# tmp1 = data.iloc[0:30,:].groupby(by=['REPORT_ID']).apply(groupDeal)
tmp1 = data.groupby(by=['REPORT_ID']).apply(groupDeal)
# tmp1
tmp1.to_csv("../add/willnorm/contest_ext_crd_cd_ln_splRaw.csv",index = False)

In [27]:
data = pd.read_csv("../add/willnorm/contest_ext_crd_cd_ln_splRaw.csv")
print(data.shape)
print(data.columns)
data.head()

(14190, 17)
Index(['REPORT_ID', 'type_dw_num', 'content_num', 'get_time_max',
       'get_time_min', 'get_time_mean', 'get_time_zongshu', 'get_time_num',
       'changing_amount_max', 'changing_amount_min', 'changing_amount_mean',
       'changing_amount_zongshu', 'changing_amount_num', 'type_dw_more',
       'type_dw_less', 'content_more', 'content_less'],
      dtype='object')


,REPORT_ID,type_dw_num,content_num,get_time_max,get_time_min,get_time_mean,get_time_zongshu,get_time_num,changing_amount_max,changing_amount_min,changing_amount_mean,changing_amount_zongshu,changing_amount_num,type_dw_more,type_dw_less,content_more,content_less
0,49099.0,2.0,2.0,1.473869e+09,1.468598e+09,1.471234e+09,1.0,2.0,2009.0,675.0,1342.000000,1.0,2.0,2.0,2.0,2.0,2.0
1,70476.0,1.0,1.0,1.473782e+09,1.473782e+09,1.473782e+09,1.0,1.0,643.0,643.0,643.000000,1.0,1.0,1.0,1.0,1.0,1.0
2,74971.0,3.0,3.0,1.478534e+09,1.468080e+09,1.474992e+09,1.0,3.0,4198.0,241.0,2412.000000,1.0,3.0,2.0,1.0,2.0,1.0
3,88203.0,7.0,7.0,1.473869e+09,1.447690e+09,1.464550e+09,4.0,7.0,3015.0,346.0,1050.571429,1.0,7.0,6.0,1.0,6.0,1.0
4,112594.0,2.0,2.0,1.475078e+09,1.465747e+09,1.470413e+09,1.0,2.0,5333.0,3000.0,4166.500000,1.0,2.0,2.0,2.0,2.0,2.0


In [28]:
# 6 个类别 *2
label =['type_dw_more', 'type_dw_less','content_more', 'content_less']
#
cate = []
c = 0
size = 0
for i in range(len(label)):
    tmp = []
    tmp = pd.get_dummies(data[label[i]],prefix = label[i])
    size += tmp.shape[1]
    if c == 0:
        cate = tmp
        c = 3
        continue
    cate = pd.concat([cate,tmp],axis = 1)
cate.head()

,type_dw_more_1.0,type_dw_more_2.0,type_dw_more_3.0,type_dw_more_4.0,type_dw_more_5.0,type_dw_more_6.0,type_dw_more_7.0,type_dw_more_8.0,type_dw_more_9.0,type_dw_more_10.0,...,content_less_109.0,content_less_116.0,content_less_117.0,content_less_118.0,content_less_122.0,content_less_178.0,content_less_180.0,content_less_235.0,content_less_255.0,content_less_325.0
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
label = data.iloc[:,1:-4].columns
# print(label)
number = data.iloc[:,1:-4]
kk = min_max_scaler.fit_transform(number)  
numbernew = pd.DataFrame(kk,columns = label)
numbernew.head()
fulldata = pd.concat([data.REPORT_ID,numbernew,cate],axis = 1)
fulldata.head()

,REPORT_ID,type_dw_num,content_num,get_time_max,get_time_min,get_time_mean,get_time_zongshu,get_time_num,changing_amount_max,changing_amount_min,...,content_less_109.0,content_less_116.0,content_less_117.0,content_less_118.0,content_less_122.0,content_less_178.0,content_less_180.0,content_less_235.0,content_less_255.0,content_less_325.0
0,49099.0,0.003086,0.003086,0.916471,0.907092,0.910439,0.00000,0.003086,0.000112,0.000038,...,0,0,0,0,0,0,0,0,0,0
1,70476.0,0.000000,0.000000,0.916235,0.920557,0.917328,0.00000,0.000000,0.000036,0.000036,...,0,0,0,0,0,0,0,0,0,0
2,74971.0,0.006173,0.006173,0.929176,0.905745,0.920598,0.00000,0.006173,0.000235,0.000013,...,0,0,0,0,0,0,0,0,0,0
3,88203.0,0.018519,0.018519,0.916471,0.852783,0.892373,0.04918,0.018519,0.000169,0.000019,...,0,0,0,0,0,0,0,0,0,0
4,112594.0,0.003086,0.003086,0.919765,0.899686,0.908220,0.00000,0.003086,0.000298,0.000168,...,0,0,0,0,0,0,0,0,0,0


In [30]:
#删掉一些维度
train = pd.read_csv("../input/contest_basic_train.tsv",sep='\t')
mm = pd.concat([train.REPORT_ID,train.Y],axis = 1)
ceshidata = fulldata.merge(mm,how = 'inner',left_on = 'REPORT_ID',right_on = 'REPORT_ID') 
print(ceshidata.head())
print(ceshidata.shape)

  REPORT_ID  type_dw_num  content_num  get_time_max  get_time_min  \
0     49099     0.003086     0.003086      0.916471      0.907092   
1     70476     0.000000     0.000000      0.916235      0.920557   
2     74971     0.006173     0.006173      0.929176      0.905745   
3     88203     0.018519     0.018519      0.916471      0.852783   
4    112594     0.003086     0.003086      0.919765      0.899686   

   get_time_mean  get_time_zongshu  get_time_num  changing_amount_max  \
0       0.910439           0.00000      0.003086             0.000112   
1       0.917328           0.00000      0.000000             0.000036   
2       0.920598           0.00000      0.006173             0.000235   
3       0.892373           0.04918      0.018519             0.000169   
4       0.908220           0.00000      0.003086             0.000298   

   changing_amount_min ...  content_less_116.0  content_less_117.0  \
0             0.000038 ...                   0                   0   
1     

In [13]:
ceshidata.Y.value_counts()

0    18384
1     1219
Name: Y, dtype: int64

In [8]:
# from imblearn.over_sampling import SMOTE, ADASYN
# X_resampled, y_resampled = SMOTE(kind='borderline1').fit_sample(ceshidata.iloc[:,1:-1], ceshidata.iloc[:,-1])

In [31]:

# print(X_resampled[0:10,:])
# print(X_resampled.shape)
# print(y_resampled[0:10])
# print(y_resampled.shape)
# pd.Series(y_resampled).value_counts()
# x_train,x_valid,y_train, y_valid= train_test_split(X_resampled,y_resampled,test_size = 0.3)
x_train,x_valid,y_train, y_valid= train_test_split(ceshidata.iloc[:,1:-1],ceshidata.iloc[:,-1],test_size = 0.3)
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(6793, 357)
(6793,)
(2912, 357)
(2912,)


In [32]:
pd.Series(y_train).value_counts()

0    6454
1     339
Name: Y, dtype: int64

In [33]:
# x_train = ceshidata.iloc[0:18000,1:-1]
# y_train = ceshidata.iloc[0:18000,-1]
# x_valid = ceshidata.iloc[18000:,1:-1]
# y_valid = ceshidata.iloc[18000:,-1]

skf = StratifiedKFold(n_splits=5)
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=3,
        scale_pos_weight=1,
        StratifiedKFold = True,
        seed=27)
xgb_param = xgb1.get_xgb_params()
#构建稀疏矩阵，运行更快
xgtrain = xgb.DMatrix(x_train, label = y_train)
# xgtest =xgb.DMatrix(dtest[predictors].values)
mm = xgb.cv(xgb_param, xgtrain, num_boost_round=1000, nfold=3,
    metrics='auc', early_stopping_rounds=50)
mm

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.501806,0.002555,0.502426,0.003430
1,0.508501,0.003450,0.552573,0.048284
2,0.523956,0.007799,0.607707,0.025758
3,0.538863,0.016386,0.645328,0.026948
4,0.541152,0.017347,0.667077,0.031956
5,0.542293,0.022538,0.676456,0.026877
6,0.550615,0.025529,0.703006,0.018699
7,0.550044,0.023996,0.711408,0.020514
8,0.545894,0.021685,0.723173,0.025403
9,0.545443,0.018284,0.740156,0.020756


In [34]:

print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(6793, 357)
(6793,)
(2912, 357)
(2912,)


In [35]:
feature = 0
for i in range(5):
    xgb2 = XGBClassifier(
            learning_rate =0.1,
            n_estimators=mm.shape[0],
            max_depth=5,
            min_child_weight=1,
            gamma=0,
            subsample=0.8,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,   #15:1
            StratifiedKFold = True,
            seed=27)
    xgb2.fit(x_train,y_train)
    feature += xgb2.feature_importances_
    print(i)
feature = feature/5

0
1
2
3
4


In [36]:
feature

array([0.03211009, 0.00642202, 0.12018348, 0.13761468, 0.12568808,
       0.03119266, 0.00275229, 0.1440367 , 0.12201835, 0.10458715,
       0.02385321, 0.        , 0.00275229, 0.00550459, 0.00733945,
       0.00275229, 0.00091743, 0.00550459, 0.00275229, 0.00275229,
       0.00091743, 0.        , 0.        , 0.00275229, 0.        ,
       0.        , 0.00183486, 0.00183486, 0.00091743, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00091743, 0.        , 0.        , 0.00091743,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [51]:
print(x_train.columns)
print(feature[feature>0.000008])
x_train.columns[feature>0.0000008].shape

Index(['type_dw_num', 'content_num', 'get_time_max', 'get_time_min',
       'get_time_mean', 'get_time_zongshu', 'get_time_num',
       'changing_amount_max', 'changing_amount_min', 'changing_amount_mean',
       ...
       'content_less_109.0', 'content_less_116.0', 'content_less_117.0',
       'content_less_118.0', 'content_less_122.0', 'content_less_178.0',
       'content_less_180.0', 'content_less_235.0', 'content_less_255.0',
       'content_less_325.0'],
      dtype='object', length=357)
[0.03211009 0.00642202 0.12018348 0.13761468 0.12568808 0.03119266
 0.00275229 0.1440367  0.12201835 0.10458715 0.02385321 0.00275229
 0.00550459 0.00733945 0.00275229 0.00091743 0.00550459 0.00275229
 0.00275229 0.00091743 0.00275229 0.00183486 0.00183486 0.00091743
 0.00091743 0.00091743 0.00458716 0.01009174 0.00275229 0.00275229
 0.01192661 0.00183486 0.00183486 0.00642202 0.00183486 0.00275229
 0.00091743 0.00275229 0.00183486 0.00183486 0.00642202 0.00091743
 0.00458716 0.00275229 0.000917

(60,)

In [52]:
fulldata.head()
fulldataok = []
fulldataok = fulldata[x_train.columns[feature>0.0000008]]  #一定要小心是谁的ID
fulldataok = pd.concat([fulldata.REPORT_ID,fulldataok],axis = 1)
fulldataok.head()

,REPORT_ID,type_dw_num,content_num,get_time_max,get_time_min,get_time_mean,get_time_zongshu,get_time_num,changing_amount_max,changing_amount_min,...,content_less_3.0,content_less_4.0,content_less_5.0,content_less_6.0,content_less_7.0,content_less_8.0,content_less_9.0,content_less_10.0,content_less_18.0,content_less_21.0
0,49099.0,0.003086,0.003086,0.916471,0.907092,0.910439,0.00000,0.003086,0.000112,0.000038,...,0,0,0,0,0,0,0,0,0,0
1,70476.0,0.000000,0.000000,0.916235,0.920557,0.917328,0.00000,0.000000,0.000036,0.000036,...,0,0,0,0,0,0,0,0,0,0
2,74971.0,0.006173,0.006173,0.929176,0.905745,0.920598,0.00000,0.006173,0.000235,0.000013,...,0,0,0,0,0,0,0,0,0,0
3,88203.0,0.018519,0.018519,0.916471,0.852783,0.892373,0.04918,0.018519,0.000169,0.000019,...,0,0,0,0,0,0,0,0,0,0
4,112594.0,0.003086,0.003086,0.919765,0.899686,0.908220,0.00000,0.003086,0.000298,0.000168,...,0,0,0,0,0,0,0,0,0,0


In [53]:
fulldataok.to_csv("../add/contest_ext_crd_cd_ln_splok.csv",index = False)

In [54]:
fulldataok.shape

(14190, 61)